# Initialization

Test notebook for the C-MAPPS benchmark. Test the Tunable Model class architecture

First we import the necessary packages and create the global variables.

In [1]:
import math
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from scipy.optimize import differential_evolution

Using TensorFlow backend.


# Create models

Create different types of models to be tested with the Tunable Model class

In [2]:
#Clear the previous tensorflow graph
K.clear_session()

FilterN = 10
FilterL = 10

def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(100, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

def RULmodel_SN_2(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULModel_CNN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    print(input_shape)
    
    #Add the layers for the model
    model.add(Conv2D(1, (FilterN, FilterL), input_shape=input_shape, padding='same', kernel_initializer='glorot_normal', activation='relu', name='cl1'))
    model.add(Dropout(0.6))
    model.add(MaxPooling2D(pool_size=5, name='pl1'))
    model.add(Flatten())
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULCNNModel(TW, FeatureN):
    
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C1')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C2')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C3')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Conv2D(1, 3, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', kernel_initializer='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    model = Model(inputs = input_layer, outputs = y, name='RUL_CNN_Model')
    
    return model

# Load the data

In [6]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

windowSize = 25
stride = 2
constRUL = 125


dataFolder = '../CMAPSSData'

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_2(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, 'keras', window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)
tModel.loadData(verbose=1, rectify_labels = False)
tModel.printData()
tModel.trimmedRUL_train

Loading data for dataset 1 with window_size of 25, stride of 2 and constRUL of 125. Cros-Validation ratio 0
Data loaded for dataset 1
Printing shapes

Training data (X, y)
(9141, 350)
(9141, 1)
Testing data (X, y)
(100, 350)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.61403509 -0.01217575 -0.39391206 ..., -0.27272727  0.38461538
   0.50126369]
 [-0.24210526 -0.10269984 -0.23156708 ..., -0.27272727  0.45299145
   0.55293457]
 [-0.22807018 -0.37480148 -0.14047351 ..., -0.27272727  0.35042735
   0.30721707]
 [-0.15087719  0.12652197 -0.52153326 ..., -0.45454545  0.36752137
   0.238697  ]
 [-0.40350877  0.05558497 -0.65546787 ..., -0.45454545  0.17948718
   0.28166245]]
[[ 125.]
 [ 125.]
 [ 125.]
 [ 125.]
 [ 125.]]
Testing data (X, y)
[[-0.41052632 -0.34886183 -0.20270575 ..., -0.27272727  0.14529915
   0.29458017]
 [ 0.44561404  0.01270513 -0.20901917 ..., -0.63636364  0.14529915
   0.0322943 ]
 [ 0.4877193  -0.02223399  0.02502818 ...,  0.09090909  0.35042735
   0.06655

array([ 67, 162,  54,  64, 144,  63, 134,  25,  76,  97, 115,  45,  38,
        55,  82,  84, 151,  70,  33, 109,  70,  77,  43,  22, 105,  74,
        31,  40,  38,  69, 109,  66,  75,  70,  56,  33,  45,  69,   3,
        63,  91,  71,  82,  67,  33, 131,  89, 106,  90,  73,  88,  88,
        70, 132,  68, 150,  12,  22, 106,  47,  60,  55,  49, 158,  28,
        77, 188,  74, 237,  12,  83,  88,  88,  41, 104,  85,  29, 106,
        74,  60, 115,  89, 168, 142,  63, 153,  53,  88,  92,  29,  10,
       216,  30, 133, 158, 211,  77,  31,  60,  75])

In [5]:
#tModel.printData(printTop=False)
#tModel.windowSize = 20
#tModel.changeDataset(2)
#tModel.loadData(verbose=1, crossValRatio=0.1)
#tModel.printData(printTop=False)
tModel.getModelDescription()
ruldcnn = RULCNNModel(tModel.windowSize, len(selected_features))
ruldcnn.summary()

Description for model: ModelRUL_SN_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 30)                10530     
_________________________________________________________________
dropout_3 (Dropout)          (None, 30)                0         
_________________________________________________________________
fc2 (Dense)                  (None, 10)                310       
_________________________________________________________________
dropout_4 (Dropout)          (None, 10)                0         
_________________________________________________________________
out (Dense)                  (None, 1)                 11        
Total params: 10,851
Trainable params: 10,851
Non-trainable params: 0
_________________________________________________________________


/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:60: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), kernel_initializer="glorot_normal", activation="tanh", name="C1", padding="same")`
/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:61: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), kernel_initializer="glorot_normal", activation="tanh", name="C2", padding="same")`
/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:62: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), kernel_initializer="glorot_normal", activation="tanh", name="C3", padding="same")`
/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:63: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), kernel_initializer="glo

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 25, 14)            0         
_________________________________________________________________
Reshape (Reshape)            (None, 25, 14, 1)         0         
_________________________________________________________________
C1 (Conv2D)                  (None, 25, 14, 10)        110       
_________________________________________________________________
C2 (Conv2D)                  (None, 25, 14, 10)        1010      
_________________________________________________________________
C3 (Conv2D)                  (None, 25, 14, 10)        1010      
_________________________________________________________________
C4 (Conv2D)                  (None, 25, 14, 10)        1010      
_________________________________________________________________
Clast (Conv2D)               (None, 25, 14, 1)         31        
__________

# Train the model and test some Tunable Model functionalities

In [8]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

#print(tModel.currentModelName)
#tModel.getAllModelsDescription()
#tModel.getModelDescription('RULModel_CNN_1')
#tModel.getModelDescription(tModel.currentModelName)
#tModel.getModelDescription()
#tModel.setCurrentModel('RULmodel_SN_1')
#tModel.getModelDescription()
tModel.epochs = 100
tModel.trainModel(learningRateScheduler=lrate, verbose=1)

Epoch 1/100
9141/9141 [==============================] - 0s 31us/step - loss: 695.3692 - mean_squared_error: 695.3692
Epoch 2/100
9141/9141 [==============================] - 0s 14us/step - loss: 689.6154 - mean_squared_error: 689.6154
Epoch 3/100
9141/9141 [==============================] - 0s 14us/step - loss: 696.6378 - mean_squared_error: 696.6378
Epoch 4/100
9141/9141 [==============================] - 0s 19us/step - loss: 704.7974 - mean_squared_error: 704.7974
Epoch 5/100
9141/9141 [==============================] - 0s 19us/step - loss: 693.1283 - mean_squared_error: 693.1283
Epoch 6/100
9141/9141 [==============================] - 0s 12us/step - loss: 691.8301 - mean_squared_error: 691.8301
Epoch 7/100
9141/9141 [==============================] - 0s 12us/step - loss: 672.7380 - mean_squared_error: 672.7380
Epoch 8/100
9141/9141 [==============================] - 0s 13us/step - loss: 660.6574 - mean_squared_error: 660.6574
Epoch 9/100
9141/9141 [==============================] -

9141/9141 [==============================] - 0s 11us/step - loss: 603.8158 - mean_squared_error: 603.8158
Epoch 70/100
9141/9141 [==============================] - 0s 10us/step - loss: 631.0898 - mean_squared_error: 631.0898
Epoch 71/100
9141/9141 [==============================] - 0s 11us/step - loss: 617.5233 - mean_squared_error: 617.5233
Epoch 72/100
9141/9141 [==============================] - 0s 11us/step - loss: 619.3109 - mean_squared_error: 619.3109
Epoch 73/100
9141/9141 [==============================] - 0s 11us/step - loss: 618.5539 - mean_squared_error: 618.5539
Epoch 74/100
9141/9141 [==============================] - 0s 11us/step - loss: 616.6912 - mean_squared_error: 616.6912
Epoch 75/100
9141/9141 [==============================] - 0s 10us/step - loss: 611.7013 - mean_squared_error: 611.7013
Epoch 76/100
9141/9141 [==============================] - 0s 11us/step - loss: 621.1200 - mean_squared_error: 621.1200
Epoch 77/100
9141/9141 [==============================] - 0s 

In [9]:
tModel.evaluateModel(["rhs"], round=2, scalingFactor=20)
print("scores")

#print(tModel.y_pred)

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.trainTime

i = 1
for prediction in tModel.y_pred:
    print('Engine {}, RUL {}, Rounded RUL {}, Real RUL {}'.format(i, prediction, 
                                                                    tModel.y_pred_rectified[i-1], 
                                                                    tModel.y_test[i-1]))
    i = i+1

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 539us/step
[0, 8.0, 0, 0, 7.0, 0, 6.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6.0, 0, 0, 0, 0, 0, 0, 0, 6.0, 0, 7.0, 0, 0, 0, 0, 0, 0, 0, 7.0, 0, 0, 9.0, 0, 11.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8.0, 7.0, 0, 7.0, 0, 0, 0, 0, 0, 10.0, 0, 6.0, 7.0, 10.0, 0, 0, 0, 0]
scores
Engine 1, RUL [ 106.57704926], Rounded RUL 106.0, Real RUL [ 112.]
Engine 2, RUL [ 112.71044159], Rounded RUL 120.0, Real RUL [ 98.]
Engine 3, RUL [ 60.17266846], Rounded RUL 60.0, Real RUL [ 69.]
Engine 4, RUL [ 74.97237396], Rounded RUL 74.0, Real RUL [ 82.]
Engine 5, RUL [ 89.8219223], Rounded RUL 96.0, Real RUL [ 91.]
Engine 6, RUL [ 92.45602417], Rounded RUL 92.0, Real RUL [ 93.]
Engine 7, RUL [ 103.26208496], Rounded RUL 109.0, Real RUL [ 91.]
Engine 8, RUL [ 74.28044891], Rounded RUL 74.0, Real RUL [ 95.]
Engine 9, RUL [ 111.99129486], Rounded RUL 111.0, Real RUL [ 111.]
Engine 10, RU

In [ ]:
def nn_optmize_fun(x, selected_features=['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32'], datasetNumber = '1', scaler = None, verbose=0, epochs=250, 
                  saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)
    windows_size = x[0]
    window_stride = x[1]
    constantRUL = x[2]
    
    if iterations == 0:
        print("Creating model")
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    
    #Define the model
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windows_size
    modelRULSN = RULmodel_SN_1(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #load the data using the selected parameters
    tModel = TunableModel(selected_features, scaler = min_max_scaler, window_stride = window_stride, 
                          window_size = windows_size, constRul = constantRUL, datasetNumber = datasetNumber, 
                         epochs=epochs)
    
    tModel.loadData()
    
    #Add the models to the tunableModel object
    tModel.addModel("RUL_SN_1_optmizable", modelRULSN)
    
    if iterations == 0:
        print("Training model")
    #Train the model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    tModel.trainCurrentModel(learningRateScheduler=lrate, verbose=verbose)
    time = tModel.getModelTimes(tModel.currentModelName)
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluateCurrentModel(["rhs"], crossValidation=True)
    cScores = tModel.getModelScores(tModel.currentModelName)
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    
    if saveToFile is not None:
        #print(msgStr)
        saveToFile.write(msgStr)
    else:
        print(msgStr)
    
    #Return RMSE as the performance metric to steer the search
    return rmse

In [ ]:
def plotRUL(cycles, rulArray, engineUnit):
    
    plt.clf()
    plt.plot(cycles, rulArray, 'bo-', label='RUL')
    plt.xlabel("Time (Cycle)")
    plt.ylabel("RUL")
    plt.title("Test Engine Unit #{}".format(engineUnit))
    
    plt.show()
    
def plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, constRUL, stride, y_test):

    df = tModel.df_test
    df_engine = df[df['Unit Number'] == engineUnit]
    
    X_test2, _, _, _ = CMAPSAuxFunctions.create_windowed_data(df_engine, selected_features, 'test', time_window=time_window, 
                                                     constRUL=constRUL, stride=stride)
    
    X_test2 = tModel.dataScaler.transform(X_test2)
    nnPred = tModel.model.predict(X_test2)
    
    maxCycle = X_test2.shape[0]
    faultCycle = tModel.y_test[engineUnit-1]
    cycles = np.arange(maxCycle)
    rulArray = np.arange(faultCycle, maxCycle+faultCycle)
    rulArray[rulArray > constRUL] = constRUL
    rulArray = np.flipud(rulArray)
    
    plotRUL(cycles, rulArray, engineUnit)
    plt.plot(cycles, nnPred, 'go-', label='NN Pred')
    plt.legend()

In [ ]:
plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, stride, constRUL)